# Agent mobility and the evolution of cooperative communities

Majeski, S. J., Linden, G., Linden, C., & Spitzer, A. (1999). Agent mobility and the evolution of cooperative communities. Complexity, 5(1), 16-24.

## Abstract

An  artificial  world  is  constructed  that  is  based  upon  a  spatial  iterated prisoner’s  dilemma  game.  Several  additional  features  are  introduced  into this  model,  the  key  feature  being  the  ability  of  agents  to  move  around  in their  world.  Movement  is  a  mechanism  for  exit  or  noncompulsory  play. When  agents  can  move,  high  levels  of  cooperation  are  achieved  more frequently  and  are  considerably  more  stable  than  when  they  cannot move.  Also,  when  cooperative  worlds  occur,  they  are  generated  and sustained  by  the  formation  of  networks  of  densely  connected “cooperative”  agents  that  can  withstand  invasion  and  parasitism  by noncooperative  agents.

In [2]:
println("Julia: $(VERSION)")
using Agents
using Random: shuffle!
using Statistics: mean
using Test: @testset, @test

Julia: 1.10.2


## 1. Define the agent

In [4]:
@enum Strategy C D

mutable struct Agent <: AbstractAgent
    id::Int
    pos::Dims{2}
    strategy::Strategy
    payoff::Float64
    
    Agent(id, pos, strategy) = new(id, pos, strategy, 0.0)
    Agent(id, pos) = Agent(id, pos, D)
    Agent() = Agent(1, (1, 1))
end

function agent_color(agent::Agent)
    if agent.strategy == C
        :blue
    elseif agent.strategy == D
        :red
    else
        error("Something wrong...")
    end
end

@testset "Agent and agent_color" begin
    agent = Agent()
    @test agent.id == 1
    @test agent.pos == (1, 1)
    @test agent.strategy == D
    @test agent.payoff == 0.0
    agent_color(agent) == :red
    
    agent.strategy = C
    agent_color(agent) == :blue
    
    agent.payoff = 9.9
    @test agent.payoff == 9.9
end;

Test Summary:         | Pass  Total  Time
Agent and agent_color |    5      5  0.1s


## 2. Define the prisoner's dilemma game (PD)

In [6]:
# Payoff Table
const PAYOFFS = (
    R = 1.0,   # CC
    T = 3.0,   # DC
    S = -3.0,  # CD
    P = -1.0   # DD
)

# Prisoner's Dilemma (PD)
function play(focal::Agent, opponent::Agent, payoffs::NamedTuple = PAYOFFS)::Tuple{Float64, Float64}
    s_pair = focal.strategy, opponent.strategy
    
    return if s_pair == (C, C)
        payoffs.R, payoffs.R
    elseif s_pair == (D, C)
        payoffs.T, payoffs.S
    elseif s_pair == (C, D)
        payoffs.S, payoffs.T
    elseif s_pair == (D, D)
        payoffs.P, payoffs.P
    else
        error("Something wrong...")
    end
end

@testset "play PD" begin
    agent1 = Agent()
    agent2 = Agent()

    agent1.strategy, agent2.strategy = (C, C)
    @test play(agent1, agent2) == (1.0, 1.0)

    agent1.strategy, agent2.strategy = (D, C)
    @test play(agent1, agent2) == (3.0, -3.0)

    agent1.strategy, agent2.strategy = (C, D)
    @test play(agent1, agent2) == (-3.0, 3.0)

    agent1.strategy, agent2.strategy = (D, D)
    @test play(agent1, agent2) == (-1.0, -1.0)
end;

Test Summary: | Pass  Total  Time
play PD       |    4      4  0.0s


## 3. Define the model

In [16]:
function build_model(; dims = (20, 20), properties)::ABM
    space = GridSpace(dims; periodic = true, metric = :chebyshev)
    model = ABM(Agent, space; properties)

    space_n = dims[1] * dims[2]
    is_agent_vec = shuffle!(vcat(fill(true, properties[:N]), fill(false, space_n - properties[:N])))
    agent_strategy_vec = shuffle!(vcat(fill(C, Int(properties[:N] / 2)), fill(D, Int(properties[:N] / 2))))
    
    space_id = 1
    agent_id = 1
    
    for x in 1:dims[1]
        for y in 1:dims[2]
            if is_agent_vec[space_id]
                add_agent_pos!(Agent(agent_id, (x, y), agent_strategy_vec[agent_id]), model)
                agent_id += 1
            end
            space_id += 1
        end
    end

    return model
end

c_rate(model::ABM)::Float64 = mean([agent.strategy == C for agent in allagents(model)])

@testset "Model" begin
    model = build_model(properties = Dict(:N => 60))

    @test nagents(model) == 60
    @test c_rate(model) == 0.5

    pos_vec = [agent.pos for agent in allagents(model)]
    for x in 1:20
        for y in 1:20
            if (x, y) ∈ pos_vec
                print("■")
            else
                print("□")
            end
        end
        print("\n")
    end
end;

□■□■□□□■□□□■□□□■■□■□
□□□□□□□■□□□□□□□■□□□□
□□□□□□■□□□□□□□□□□□□□
■□□□□□□□■□□□■■□■□■□□
□□□□□□■□□□□■□□□□□■□□
□□□□□□□□□□■□□□□□□□□■
□□□□□□□□□□□□□□■□□□□□
□□■□□□■□■□□□□□□□□■□□
□□□□■■□□■□□□□□■□□□□□
□■□□□□□□■□□□■□□□□□□□
□□■□□□□□□□■□□□■□□□□□
□□□□□□■□□□□□□□□■□□□□
□□□■□□□□□□□□□□□□□□□□
□□□□□□□□□■□□□□□□□□□□
□■□□■□□□■□■□■□□□□□■■
□□□■□■□■□■■□□□□□□□□□
□□□□□□■□□□□■□□□■□□□□
□□□□□□□□□□□□□□□□□□□□
□□□□□□□□□□□□□□□□■□□□
□□□■□□■■□□□□□□□□■□□□
Test Summary: | Pass  Total  Time
Model         |    2      2  0.1s


In [9]:
function model_step!(model)
    agents_done = []

    for focal_agent in allagents(model)
        opponent_agent = random_nearby_agent(focal_agent, model)
        focal_payoff, opponent_payoff = play(focal_agent, opponent_agent)
        focal_agent.payoff += focal_payoff
        opponent_agent.payoff += opponent_payoff

        push!(agents_done, focal_agent.id)
        push!(agents_done, opponent_agent.id)

        # ToDo: すでにゲーム済みのエージェントが二回目のゲームをプレイしないようにする
    end

    # At the start of the next generation, each lattice-site is occupied by the player with the highest score among the previous owner and the immediate neighbours.
    for agent in allagents(model)
        best_payoff = agent.payoff
        agent.next_strategy = agent.strategy

        for neighbor in nearby_agents(agent, model)
            if neighbor.payoff > best_payoff
                best_payoff = neighbor.payoff
                agent.next_strategy = neighbor.strategy
            end
        end
    end

    # update strategy
    for agent in allagents(model)
        agent.prev_strategy = agent.strategy
        agent.strategy = agent.next_strategy
    end
end

StandardABM with 60 agents of type Agent
 agents container: Dict
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest
 properties: N